[View in Colaboratory](https://colab.research.google.com/github/santhavathi/Deep-Learning-MLBLR/blob/master/DenseNet_CIFAR10_Final.ipynb)

In [0]:
import numpy as np

In [3]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 50
l = 40
num_filter = 12
compression = 0.5
dropout_rate = 0.2

In [0]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [0]:
#rescale=1./255

#train_datagen = ImageDataGenerator(
#        featurewise_center=False,  # set input mean to 0 over the dataset
#        samplewise_center=False,  # set each sample mean to 0
#        featurewise_std_normalization=False,  # divide inputs by std of the dataset
#        samplewise_std_normalization=False,  # divide each input by its std
#        zca_whitening=False,  # apply ZCA whitening
#        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
#        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
#        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
#        horizontal_flip=True,  # randomly flip images
#        vertical_flip=False,
#        rescale=rescale)  # randomly flip images

#datagen = ImageDataGenerator(rescale=rescale)
#datagen.fit(x_train)
#datagen.fit(x_test)

In [0]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [0]:
# Dense Block
def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [12]:
num_filter = 64
dropout_rate = 0.4
l = 14

input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

num_filter = 32
dropout_rate = 0.2
l = 14

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)

model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 64)   1728        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 64)   256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 64)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

conv2d_54 (Conv2D)              (None, 4, 4, 16)     20736       activation_53[0][0]              
__________________________________________________________________________________________________
dropout_53 (Dropout)            (None, 4, 4, 16)     0           conv2d_54[0][0]                  
__________________________________________________________________________________________________
concatenate_50 (Concatenate)    (None, 4, 4, 160)    0           concatenate_49[0][0]             
                                                                 dropout_53[0][0]                 
__________________________________________________________________________________________________
batch_normalization_54 (BatchNo (None, 4, 4, 160)    640         concatenate_50[0][0]             
__________________________________________________________________________________________________
activation_54 (Activation)      (None, 4, 4, 160)    0           batch_normalization_54[0][0]     
__________

In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [0]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

for i in range(1,6):
  model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_test))
  
  #train_datagen = model.fit_generator(datagen.flow(x_train, y_train,
  #                                 batch_size=batch_size),
  #                                 epochs=epochs,
  #                                 validation_data=(x_test, y_test),
  #                                 workers=4)


  file_name = 'DenseNet_CIFAR10_' + str(i) + '.h5'

  # Save the trained weights in to .h5 format
  #model.save(file_name)
  model.save_weights(file_name)
  print("Saved model to disk - {0}".format(file_name))

  # Saving using PyDrive to google drive

  # Create & upload a file.
  uploaded = drive.CreateFile({'title': file_name})
  uploaded.SetContentFile(file_name)
  uploaded.Upload()
  print('Uploaded file with ID {}'.format(uploaded.get('id')))

Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 645s 13ms/step - loss: 1.5357 - acc: 0.4383 - val_loss: 2.6936 - val_acc: 0.3320
Epoch 2/10
19072/50000 [==========>...................] - ETA: 5:54 - loss: 1.1724 - acc: 0.5749

50000/50000 [==============================] - 618s 12ms/step - loss: 1.0875 - acc: 0.6073 - val_loss: 1.1640 - val_acc: 0.6289
Epoch 3/10
45184/50000 [==========================>...] - ETA: 55s - loss: 0.9162 - acc: 0.6717

50000/50000 [==============================] - 619s 12ms/step - loss: 0.9116 - acc: 0.6732 - val_loss: 1.3001 - val_acc: 0.6247
Epoch 4/10
50000/50000 [==============================] - 618s 12ms/step - loss: 0.7971 - acc: 0.7140 - val_loss: 2.5836 - val_acc: 0.4419
Epoch 5/10
 2944/50000 [>.............................] - ETA: 8:59 - loss: 0.7184 - acc: 0.7401

50000/50000 [==============================] - 618s 12ms/step - loss: 0.7073 - acc: 0.7508 - val_loss: 1.7575 - val_acc: 0.6080
Epoch 6/10
39040/50000 [======================>.......] - ETA: 2:05 - loss: 0.6420 - acc: 0.7724

50000/50000 [==============================] - 618s 12ms/step - loss: 0.6398 - acc: 0.7738 - val_loss: 1.2379 - val_acc: 0.6776
Epoch 7/10
50000/50000 [==============================] - 618s 12ms/step - loss: 0.5961 - acc: 0.7905 - val_loss: 1.4388 - val_acc: 0.6428
Epoch 8/10
 1536/50000 [..............................] - ETA: 9:15 - loss: 0.5326 - acc: 0.8314

50000/50000 [==============================] - 618s 12ms/step - loss: 0.5436 - acc: 0.8078 - val_loss: 1.7878 - val_acc: 0.6027
Epoch 9/10
38528/50000 [======================>.......] - ETA: 2:11 - loss: 0.5193 - acc: 0.8167

50000/50000 [==============================] - 619s 12ms/step - loss: 0.5207 - acc: 0.8172 - val_loss: 1.3701 - val_acc: 0.6540
Epoch 10/10
50000/50000 [==============================] - 618s 12ms/step - loss: 0.4869 - acc: 0.8292 - val_loss: 1.4594 - val_acc: 0.6463
Saved model to disk - DenseNet_CIFAR10_1.h5
Uploaded file with ID 1mVgcnNODIrzm1zAIskBIpbm0d9mMXKIm
Train on 50000 samples, validate on 10000 samples
Epoch 1/10
  768/50000 [..............................] - ETA: 9:31 - loss: 0.4816 - acc: 0.8229

50000/50000 [==============================] - 619s 12ms/step - loss: 0.4584 - acc: 0.8401 - val_loss: 1.1230 - val_acc: 0.7048
Epoch 2/10
38272/50000 [=====================>........] - ETA: 2:14 - loss: 0.4401 - acc: 0.8464

50000/50000 [==============================] - 619s 12ms/step - loss: 0.4384 - acc: 0.8476 - val_loss: 1.2403 - val_acc: 0.7123
Epoch 3/10
32640/50000 [==================>...........] - ETA: 3:18 - loss: 0.4121 - acc: 0.8559

In [13]:
!ls -l

total 27828
drwxr-xr-x 1 root root     4096 May 12 10:05 datalab
-rw-r--r-- 1 root root 14244496 May 12 13:32 DenseNet_CIFAR10_1.h5
-rw-r--r-- 1 root root 14244496 May 12 15:59 DenseNet_CIFAR10_2.h5


In [0]:
model.load_weights('DenseNet_CIFAR10_2.h5')

In [0]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

for i in range(2,6):
  model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_test))
  
  #train_datagen = model.fit_generator(datagen.flow(x_train, y_train,
  #                                 batch_size=batch_size),
  #                                 epochs=epochs,
  #                                 validation_data=(x_test, y_test),
  #                                 workers=4)


  file_name = 'DenseNet_CIFAR10_' + str(i) + '.h5'

  # Save the trained weights in to .h5 format
  #model.save(file_name)
  model.save_weights(file_name)
  print("Saved model to disk - {0}".format(file_name))

  # Saving using PyDrive to google drive

  # Create & upload a file.
  uploaded = drive.CreateFile({'title': file_name})
  uploaded.SetContentFile(file_name)
  uploaded.Upload()
  print('Uploaded file with ID {}'.format(uploaded.get('id')))

Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 640s 13ms/step - loss: 0.4726 - acc: 0.8339 - val_loss: 0.8854 - val_acc: 0.7628
Epoch 2/10
19072/50000 [==========>...................] - ETA: 5:53 - loss: 0.4495 - acc: 0.8406

50000/50000 [==============================] - 617s 12ms/step - loss: 0.4499 - acc: 0.8419 - val_loss: 1.0399 - val_acc: 0.7294
Epoch 3/10
45184/50000 [==========================>...] - ETA: 55s - loss: 0.4157 - acc: 0.8524

50000/50000 [==============================] - 617s 12ms/step - loss: 0.4164 - acc: 0.8522 - val_loss: 0.8201 - val_acc: 0.7854
Epoch 4/10
50000/50000 [==============================] - 617s 12ms/step - loss: 0.3980 - acc: 0.8609 - val_loss: 0.8160 - val_acc: 0.7733
Epoch 5/10
 2944/50000 [>.............................] - ETA: 8:58 - loss: 0.3736 - acc: 0.8679

50000/50000 [==============================] - 617s 12ms/step - loss: 0.3818 - acc: 0.8659 - val_loss: 1.0340 - val_acc: 0.7450
Epoch 6/10
39040/50000 [======================>.......] - ETA: 2:05 - loss: 0.3599 - acc: 0.8744

50000/50000 [==============================] - 617s 12ms/step - loss: 0.3614 - acc: 0.8730 - val_loss: 1.3355 - val_acc: 0.6973
Epoch 7/10
50000/50000 [==============================] - 617s 12ms/step - loss: 0.3429 - acc: 0.8800 - val_loss: 1.2650 - val_acc: 0.6960
Epoch 8/10
 1536/50000 [..............................] - ETA: 9:13 - loss: 0.3274 - acc: 0.8867

50000/50000 [==============================] - 617s 12ms/step - loss: 0.3254 - acc: 0.8844 - val_loss: 0.7819 - val_acc: 0.8083
Epoch 9/10
38528/50000 [======================>.......] - ETA: 2:11 - loss: 0.3133 - acc: 0.8897

50000/50000 [==============================] - 617s 12ms/step - loss: 0.3156 - acc: 0.8893 - val_loss: 1.4855 - val_acc: 0.6664
Epoch 10/10
36736/50000 [=====================>........] - ETA: 2:31 - loss: 0.3007 - acc: 0.8935

###Score after 20 epochs

In [16]:
score = model.evaluate(x_test, y_test, verbose=1)
print(score)

10000/10000 [==============================] - 57s 6ms/step
[1.0432363877296447, 0.7534]


In [17]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

for i in range(3,6):
  model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_test))
  
  #train_datagen = model.fit_generator(datagen.flow(x_train, y_train,
  #                                 batch_size=batch_size),
  #                                 epochs=epochs,
  #                                 validation_data=(x_test, y_test),
  #                                 workers=4)


  file_name = 'DenseNet_CIFAR10_' + str(i) + '.h5'

  # Save the trained weights in to .h5 format
  #model.save(file_name)
  model.save_weights(file_name)
  print("Saved model to disk - {0}".format(file_name))

  # Saving using PyDrive to google drive

  # Create & upload a file.
  uploaded = drive.CreateFile({'title': file_name})
  uploaded.SetContentFile(file_name)
  uploaded.Upload()
  print('Uploaded file with ID {}'.format(uploaded.get('id')))

Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 642s 13ms/step - loss: 0.2996 - acc: 0.8955 - val_loss: 1.9391 - val_acc: 0.6511
Epoch 2/10
19072/50000 [==========>...................] - ETA: 5:54 - loss: 0.2849 - acc: 0.8994

50000/50000 [==============================] - 619s 12ms/step - loss: 0.2875 - acc: 0.8982 - val_loss: 0.6840 - val_acc: 0.8218
Epoch 3/10
45184/50000 [==========================>...] - ETA: 55s - loss: 0.2721 - acc: 0.9043

50000/50000 [==============================] - 619s 12ms/step - loss: 0.2727 - acc: 0.9044 - val_loss: 0.8907 - val_acc: 0.8000
Epoch 4/10
50000/50000 [==============================] - 619s 12ms/step - loss: 0.2608 - acc: 0.9087 - val_loss: 0.9232 - val_acc: 0.7938
Epoch 5/10
 2944/50000 [>.............................] - ETA: 8:59 - loss: 0.2421 - acc: 0.9141

50000/50000 [==============================] - 619s 12ms/step - loss: 0.2515 - acc: 0.9112 - val_loss: 0.5231 - val_acc: 0.8678
Epoch 6/10
39168/50000 [======================>.......] - ETA: 2:04 - loss: 0.2388 - acc: 0.9154

50000/50000 [==============================] - 619s 12ms/step - loss: 0.2414 - acc: 0.9144 - val_loss: 0.7384 - val_acc: 0.8234
Epoch 7/10
50000/50000 [==============================] - 619s 12ms/step - loss: 0.2382 - acc: 0.9146 - val_loss: 0.8782 - val_acc: 0.8093
Epoch 8/10
 1536/50000 [..............................] - ETA: 9:17 - loss: 0.1832 - acc: 0.9382

...

50000/50000 [==============================] - 619s 12ms/step - loss: 0.1747 - acc: 0.9383 - val_loss: 1.4080 - val_acc: 0.7403
Epoch 6/10
50000/50000 [==============================] - 619s 12ms/step - loss: 0.1737 - acc: 0.9384 - val_loss: 0.7417 - val_acc: 0.8339
Epoch 7/10
 1408/50000 [..............................] - ETA: 9:16 - loss: 0.1596 - acc: 0.9411

50000/50000 [==============================] - 619s 12ms/step - loss: 0.1691 - acc: 0.9396 - val_loss: 0.8341 - val_acc: 0.8308
Epoch 8/10
38528/50000 [======================>.......] - ETA: 2:11 - loss: 0.1633 - acc: 0.9414

50000/50000 [==============================] - 619s 12ms/step - loss: 0.1646 - acc: 0.9411 - val_loss: 0.6393 - val_acc: 0.8518
Epoch 9/10
50000/50000 [==============================] - 619s 12ms/step - loss: 0.1560 - acc: 0.9437 - val_loss: 0.7381 - val_acc: 0.8480
Epoch 10/10
 1408/50000 [..............................] - ETA: 9:18 - loss: 0.1352 - acc: 0.9553

50000/50000 [==============================] - 619s 12ms/step - loss: 0.1541 - acc: 0.9446 - val_loss: 0.6435 - val_acc: 0.8644
Saved model to disk - DenseNet_CIFAR10_4.h5


InvalidConfigError: ignored

In [14]:
from google.colab import files
files.upload()

Saving DenseNet_CIFAR10_3 (1).h5 to DenseNet_CIFAR10_3 (1).h5


In [0]:
model.load_weights('DenseNet_CIFAR10_3 (1).h5')

###Score after 30 epochs

In [15]:
score = model.evaluate(x_test, y_test, verbose=1)
print(score)

10000/10000 [==============================] - 58s 6ms/step
[0.6218343338787555, 0.845]


In [16]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

for i in range(4,6):
  model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_test))
  
  #train_datagen = model.fit_generator(datagen.flow(x_train, y_train,
  #                                 batch_size=batch_size),
  #                                 epochs=epochs,
  #                                 validation_data=(x_test, y_test),
  #                                 workers=4)


  file_name = 'DenseNet_CIFAR10_' + str(i) + '.h5'

  # Save the trained weights in to .h5 format
  #model.save(file_name)
  model.save_weights(file_name)
  print("Saved model to disk - {0}".format(file_name))

  # Saving using PyDrive to google drive

  # Create & upload a file.
  uploaded = drive.CreateFile({'title': file_name})
  uploaded.SetContentFile(file_name)
  uploaded.Upload()
  print('Uploaded file with ID {}'.format(uploaded.get('id')))

Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 651s 13ms/step - loss: 0.2084 - acc: 0.9256 - val_loss: 0.7997 - val_acc: 0.8258
Epoch 2/10
19072/50000 [==========>...................] - ETA: 5:59 - loss: 0.1921 - acc: 0.9319

50000/50000 [==============================] - 629s 13ms/step - loss: 0.1972 - acc: 0.9303 - val_loss: 1.0086 - val_acc: 0.7824
Epoch 3/10
45184/50000 [==========================>...] - ETA: 56s - loss: 0.1848 - acc: 0.9343

50000/50000 [==============================] - 629s 13ms/step - loss: 0.1864 - acc: 0.9341 - val_loss: 0.9619 - val_acc: 0.8061
Epoch 4/10
50000/50000 [==============================] - 629s 13ms/step - loss: 0.1872 - acc: 0.9334 - val_loss: 0.6336 - val_acc: 0.8542
Epoch 5/10
 2944/50000 [>.............................] - ETA: 9:07 - loss: 0.1622 - acc: 0.9406

50000/50000 [==============================] - 629s 13ms/step - loss: 0.1765 - acc: 0.9367 - val_loss: 0.9155 - val_acc: 0.8085
Epoch 6/10
39040/50000 [======================>.......] - ETA: 2:07 - loss: 0.1716 - acc: 0.9393

50000/50000 [==============================] - 629s 13ms/step - loss: 0.1760 - acc: 0.9373 - val_loss: 1.0458 - val_acc: 0.7986
Epoch 7/10
50000/50000 [==============================] - 625s 12ms/step - loss: 0.1651 - acc: 0.9415 - val_loss: 1.1058 - val_acc: 0.7862
Epoch 8/10
 1536/50000 [..............................] - ETA: 9:19 - loss: 0.1607 - acc: 0.9479

...

50000/50000 [==============================] - 626s 13ms/step - loss: 0.1281 - acc: 0.9542 - val_loss: 0.7224 - val_acc: 0.8562
Epoch 6/10
50000/50000 [==============================] - 626s 13ms/step - loss: 0.1301 - acc: 0.9537 - val_loss: 0.6436 - val_acc: 0.8650
Epoch 7/10
 1408/50000 [..............................] - ETA: 9:23 - loss: 0.1262 - acc: 0.9588

50000/50000 [==============================] - 625s 13ms/step - loss: 0.1284 - acc: 0.9547 - val_loss: 0.7621 - val_acc: 0.8530
Epoch 8/10
38528/50000 [======================>.......] - ETA: 2:12 - loss: 0.1161 - acc: 0.9583

50000/50000 [==============================] - 625s 13ms/step - loss: 0.1187 - acc: 0.9576 - val_loss: 0.6790 - val_acc: 0.8564
Epoch 9/10
50000/50000 [==============================] - 625s 12ms/step - loss: 0.1156 - acc: 0.9578 - val_loss: 0.6604 - val_acc: 0.8600
Epoch 10/10
 1408/50000 [..............................] - ETA: 9:21 - loss: 0.1184 - acc: 0.9624

50000/50000 [==============================] - 626s 13ms/step - loss: 0.1151 - acc: 0.9593 - val_loss: 0.5517 - val_acc: 0.8814
Saved model to disk - DenseNet_CIFAR10_5.h5


InvalidConfigError: ignored

###Score after 50 epochs

In [17]:
score = model.evaluate(x_test, y_test, verbose=1)
print(score)

10000/10000 [==============================] - 55s 5ms/step
[0.5516593681804836, 0.8814]


In [20]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

uploaded = drive.CreateFile({'title': 'DenseNet_CIFAR10_5.h5'})
uploaded.SetContentFile('DenseNet_CIFAR10_5.h5')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

Uploaded file with ID 1vtHTEaPBn_cn6wqJEXFYcBVYhqeFdf71
